In [1]:
import pandas
from pypdf import PdfReader, PdfWriter
from pypdf.errors import PdfStreamError
from ocrmypdf.exceptions import SubprocessOutputError
import os
import ocrmypdf
from tqdm import tqdm
from IPython.display import clear_output
import json
from unidecode import unidecode
import re
import traceback
from collections import defaultdict

In [4]:
skip_files = []
files_with_error = []
processed_files = []

raw_dir = './files/raw'
tmp_dir = './files/tmp'
output_dir = './files/ocr_address'
json_dir = './files/json'
sample_dir = './files/sample'
log_dir = './var'

In [ ]:
for dirname in [raw_dir, tmp_dir, output_dir, json_dir, log_dir]:
    if not os.path.exists(dirname):
        os.makedirs(dirname)

In [3]:
for name in os.listdir(raw_dir):
    src = os.path.join(raw_dir, name)
    reader = PdfReader(src)
        
    if len(reader.pages) <= 4:
        skip_files.append(name)
        continue
    
    dest = os.path.join(tmp_dir, name)
    
    #Extraer la penúltima página, donde se encuentra la dirección de la estación
    
    with open (os.path.join(tmp_dir, name), 'wb') as f:
        writer = PdfWriter(f)
        writer.add_page(reader.pages[reader.get_num_pages()-2])
        writer.write(dest)

/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in the trailer - assuming there is no previous xref table
/Prev=0 in

In [29]:
def extract_address(file: PdfReader) -> str|None:
    pattern = r'E/S\s+(.*?)(?=\n(?:Precios|\*recios|MEDICIÓN|COMPARACIÓN|PROCESO|VISTA|MUESTRAS|TANQUES|SERVICIO|$))'
    page = file.pages[0]
    # for page in file.pages:
    page_text = page.extract_text()
    match = re.search(pattern, page_text, re.DOTALL)
    
    if match:
       return match.group(1)
    else:
        return None
    


In [30]:
no_match = []
matches = []
for name in os.listdir(output_dir):
    file = PdfReader(os.path.join(output_dir, name))
    address = extract_address(file)
    
    if address is None:
        no_match.append(name)
        continue
    matches.append({'file': name, 'address': address})
        
    

In [31]:
len(matches)

657

In [32]:
no_match[:5]

['2023 03 140.pdf',
 '2023 06 005.pdf',
 '2023 05 065.pdf',
 '2023 08 047.pdf',
 '2023 05 064.pdf']

In [36]:
matches[10:25]

[{'file': '2023 05 105.pdf',
  'address': 'NEXT NUEVOSOL\nAV. JACOBO  MAJLUTA+F14,  SANTO DOMINGO  NORTE'},
 {'file': '2023 03 034.pdf',
  'address': 'TEXACO PABLO  ILSECARRETERA  MOCASAN  VICTOR  CRUCE  ESTANCIA  NUEVA  MOCA,  PROVINCIA  ESPAILLATq¿_ _ _ A\nsi\nRD$29360 RD$27450\nPapa\nRDS24110. — RDS2216'},
 {'file': '2023 03 008.pdf',
  'address': 'SHELL  MABAZ\nAVENIDA  SANTA  ROSA,  ESQUINA  DUARTE, PROVINCIA  LA ROMANA'},
 {'file': '2023 08 052.pdf',
  'address': 'SHELL  RAE - AUTOPISTA  30 DE MAYO,  KM. 11 1/2, MARENSA.  DISTRITO  NACIONAL'},
 {'file': '2023 05 071.pdf',
  'address': 'ECOPETROLEO  SANTA — CARRETERA  BARAHONA - PARAISO,  LA CIENAGA,  PROVINCIA\nBARAHONA\nUT'},
 {'file': '2023 08 046.pdf',
  'address': 'PETROMÓVIL  MERCA\nKM 20, AUTOPISTA  DUARTE  AL SUR DEL MERCA  SANTO  DOMINGO  — SANTO DOMINGO'},
 {'file': '2023 05 059.pdf',
  'address': 'MEDICIÓN  DEL CORTE  DE AGUA LIBRE  EN LOS'},
 {'file': '2023 02 025.pdf',
  'address': 'TEXACO SAN ANDRES\nAUTOPISTA  LAS A

In [5]:
import warnings

error_files = []
warning_files = []

for name in tqdm(os.listdir(tmp_dir), unit='file', desc='Files processed: '):
    
    #Omite los archivos que tienen cuatro páginas o menos
    if name in skip_files or os.path.exists(os.path.join(output_dir, name)):
        clear_output(wait=True)
        continue
    
    try:
        with warnings.catch_warnings(record = True) as w:
            
            warnings.simplefilter("always")
            #Realiza OCR
            ocrmypdf.ocr(os.path.join(tmp_dir, name),
                    os.path.join(output_dir, name),skip_text=True, language='spa', deskew=True)
            if w:
                raise w[-1].category(w[-1].message)
    
    except SubprocessOutputError as error:
        error_files.append({'file': name, 'error': error})
    
    except Warning as e:
        warning_files.append({'file': name, 'warning': e})
        
    clear_output(wait=True)

Files processed: 100%|██████████| 810/810 [1:53:22<00:00,  8.40s/file]
